Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [31]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in `1_notmnist.ipynb`.

In [32]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [33]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [34]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

## Logistic Regression w/ L2 Regularization

In [35]:
batch_size = 128
beta = 0.01 # value to tune for L2 Regularization

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Calculate score (i.e. logits)
  logits = tf.matmul(tf_train_dataset, weights) + biases

  # Calculate loss (w/ L2 Regularization)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
  regularizer = tf.nn.l2_loss(weights)
  loss = tf.reduce_mean(loss + beta * regularizer) # apply L2 regularization to loss
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

Now we run it:

In [36]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 50.330002
Minibatch accuracy: 9.4%
Validation accuracy: 11.7%
Minibatch loss at step 500: 0.723955
Minibatch accuracy: 86.7%
Validation accuracy: 81.5%
Minibatch loss at step 1000: 0.789225
Minibatch accuracy: 78.1%
Validation accuracy: 81.3%
Minibatch loss at step 1500: 0.582886
Minibatch accuracy: 84.4%
Validation accuracy: 81.2%
Minibatch loss at step 2000: 0.672778
Minibatch accuracy: 87.5%
Validation accuracy: 80.9%
Minibatch loss at step 2500: 0.707022
Minibatch accuracy: 81.2%
Validation accuracy: 80.5%
Minibatch loss at step 3000: 0.856284
Minibatch accuracy: 79.7%
Validation accuracy: 81.5%
Test accuracy: 88.5%


## Neural Network w/ L2 Regularization

In [37]:
batch_size = 128
beta = 0.01 # value to tune for L2 regularization

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights_1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, 1024]))
  biases_1 = tf.Variable(tf.zeros([1024]))
  weights_2 = tf.Variable(
    tf.truncated_normal([1024, num_labels]))
  biases_2 = tf.Variable(tf.zeros([num_labels]))

  # Hidden layer
  hidden_layer = tf.nn.relu(tf.matmul(tf_train_dataset, weights_1) + biases_1)
  
  # Training computation.
  logits = tf.matmul(hidden_layer, weights_2) + biases_2

  # Calculate loss (w/ L2 Regularization)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
  regularizer_1 = tf.nn.l2_loss(weights_1)
  regularizer_2 = tf.nn.l2_loss(weights_2)
  loss = tf.reduce_mean(loss + beta * (regularizer_1 + regularizer_2)) # apply L2 regularization to loss
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training data
  train_prediction = tf.nn.softmax(logits)

  # Predictions for the validation data
  valid_hidden_layer = tf.nn.relu(tf.matmul(tf_valid_dataset, weights_1) + biases_1)
  valid_logits = tf.matmul(valid_hidden_layer, weights_2) + biases_2
  valid_prediction = tf.nn.softmax(valid_logits)

  # Predictions for the testing data
  test_hidden_layer = tf.nn.relu(tf.matmul(tf_test_dataset, weights_1) + biases_1)
  test_logits = tf.matmul(test_hidden_layer, weights_2) + biases_2
  test_prediction = tf.nn.softmax(test_logits)

Now let's run it:

In [38]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3485.519775
Minibatch accuracy: 4.7%
Validation accuracy: 28.4%
Minibatch loss at step 500: 21.164518
Minibatch accuracy: 88.3%
Validation accuracy: 84.2%
Minibatch loss at step 1000: 0.942327
Minibatch accuracy: 82.0%
Validation accuracy: 83.3%
Minibatch loss at step 1500: 0.595085
Minibatch accuracy: 88.3%
Validation accuracy: 83.4%
Minibatch loss at step 2000: 0.612253
Minibatch accuracy: 90.6%
Validation accuracy: 83.3%
Minibatch loss at step 2500: 0.641817
Minibatch accuracy: 85.2%
Validation accuracy: 83.2%
Minibatch loss at step 3000: 0.829997
Minibatch accuracy: 82.0%
Validation accuracy: 83.7%
Test accuracy: 90.4%


---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [39]:
num_steps = 3001

# restrict training set to induce overfitting
restricted_train_dataset = train_dataset[:1000, :]
restricted_train_labels = train_labels[:1000]

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (restricted_train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = restricted_train_dataset[offset:(offset + batch_size), :]
    batch_labels = restricted_train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3511.693604
Minibatch accuracy: 10.9%
Validation accuracy: 20.2%
Minibatch loss at step 500: 21.059305
Minibatch accuracy: 100.0%
Validation accuracy: 79.6%
Minibatch loss at step 1000: 0.557882
Minibatch accuracy: 100.0%
Validation accuracy: 80.2%
Minibatch loss at step 1500: 0.387576
Minibatch accuracy: 100.0%
Validation accuracy: 79.8%
Minibatch loss at step 2000: 0.385305
Minibatch accuracy: 100.0%
Validation accuracy: 80.1%
Minibatch loss at step 2500: 0.452998
Minibatch accuracy: 97.7%
Validation accuracy: 79.1%
Minibatch loss at step 3000: 0.357377
Minibatch accuracy: 100.0%
Validation accuracy: 80.4%
Test accuracy: 87.8%


---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [40]:
batch_size = 128
beta = 0.01 # value to tune for L2 regularization

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights_1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, 1024]))
  biases_1 = tf.Variable(tf.zeros([1024]))
  weights_2 = tf.Variable(
    tf.truncated_normal([1024, num_labels]))
  biases_2 = tf.Variable(tf.zeros([num_labels]))
    
  first_layer_logits = tf.matmul(tf_train_dataset, weights_1) + biases_1

  # Hidden layer
  hidden_layer = tf.nn.relu(first_layer_logits)

  # Apply dropout to hidden layer
  keep_prob = tf.placeholder("float")
  hidden_layer = tf.nn.dropout(hidden_layer, keep_prob)
  
  # Training computation.
  logits = tf.matmul(hidden_layer, weights_2) + biases_2

  # Calculate loss (w/ L2 Regularization)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
  regularizer_1 = tf.nn.l2_loss(weights_1)
  regularizer_2 = tf.nn.l2_loss(weights_2)
  loss = tf.reduce_mean(loss + beta * (regularizer_1 + regularizer_2)) # apply L2 regularization to loss
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training data
  train_prediction = tf.nn.softmax(logits)

  # Predictions for the validation data
  valid_hidden_layer = tf.nn.relu(tf.matmul(tf_valid_dataset, weights_1) + biases_1)
  valid_logits = tf.matmul(valid_hidden_layer, weights_2) + biases_2
  valid_prediction = tf.nn.softmax(valid_logits)

  # Predictions for the testing data
  test_hidden_layer = tf.nn.relu(tf.matmul(tf_test_dataset, weights_1) + biases_1)
  test_logits = tf.matmul(test_hidden_layer, weights_2) + biases_2
  test_prediction = tf.nn.softmax(test_logits)

Note that we set the `keep_prob` value to 0.5 in the `feed_dict` dictionary.

Now let's run it on an un-restricted training set:

In [44]:
num_steps = 6001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, keep_prob : 0.5}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3636.897217
Minibatch accuracy: 10.2%
Validation accuracy: 24.6%
Minibatch loss at step 500: 21.406073
Minibatch accuracy: 84.4%
Validation accuracy: 83.7%
Minibatch loss at step 1000: 1.013307
Minibatch accuracy: 82.0%
Validation accuracy: 82.6%
Minibatch loss at step 1500: 0.688930
Minibatch accuracy: 85.9%
Validation accuracy: 83.2%
Minibatch loss at step 2000: 0.708478
Minibatch accuracy: 86.7%
Validation accuracy: 83.1%
Minibatch loss at step 2500: 0.710290
Minibatch accuracy: 82.8%
Validation accuracy: 83.1%
Minibatch loss at step 3000: 0.895955
Minibatch accuracy: 80.5%
Validation accuracy: 83.0%
Minibatch loss at step 3500: 0.762235
Minibatch accuracy: 82.0%
Validation accuracy: 83.1%
Minibatch loss at step 4000: 0.764581
Minibatch accuracy: 84.4%
Validation accuracy: 83.2%
Minibatch loss at step 4500: 0.731464
Minibatch accuracy: 83.6%
Validation accuracy: 83.0%
Minibatch loss at step 5000: 0.732971
Minibatch accuracy: 83.6%
Validation acc

Now let's try this with the restricted training set:

In [29]:
num_steps = 3001

# restrict training set to induce overfitting
restricted_train_dataset = train_dataset[:1000, :]
restricted_train_labels = train_labels[:1000]

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (restricted_train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = restricted_train_dataset[offset:(offset + batch_size), :]
    batch_labels = restricted_train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, keep_prob : 0.5}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3616.107910
Minibatch accuracy: 5.5%
Validation accuracy: 27.7%
Minibatch loss at step 500: 21.061281
Minibatch accuracy: 100.0%
Validation accuracy: 75.7%
Minibatch loss at step 1000: 0.429076
Minibatch accuracy: 100.0%
Validation accuracy: 76.4%
Minibatch loss at step 1500: 0.261679
Minibatch accuracy: 100.0%
Validation accuracy: 76.8%
Minibatch loss at step 2000: 0.242850
Minibatch accuracy: 100.0%
Validation accuracy: 76.8%
Minibatch loss at step 2500: 0.240391
Minibatch accuracy: 100.0%
Validation accuracy: 77.0%
Minibatch loss at step 3000: 0.242146
Minibatch accuracy: 100.0%
Validation accuracy: 76.9%
Test accuracy: 84.1%


---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---
